<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
#!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

In [3]:
articles_df = pd.read_csv("../input/articles-sharing-reading-from-cit-deskdrop/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [4]:
interactions_df = pd.read_csv("../input/articles-sharing-reading-from-cit-deskdrop/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [6]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [8]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [9]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [10]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [11]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [12]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации будем пользоваться метрикой *precision@10*.

In [13]:
#!pip install lightfm

In [14]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.data import Dataset

Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [15]:
import tensorflow
tensorflow.random.set_seed(10)

In [16]:
#https://making.lyst.com/lightfm/docs/lightfm.data.html взяла построение датасета из документации модели, которая потом предлагается для использования
dataset = Dataset()
dataset.fit(interactions_full_df.personId, interactions_full_df.contentId)

X = (interactions_train_df[["personId", "contentId", "eventStrength"]].apply(tuple,axis=1),interactions_test_df[["personId", "contentId", "eventStrength"]].apply(tuple,axis=1))

In [17]:
data_train = dataset.build_interactions(X[0])[1] #беру из массива выше первую часть, соответствующую трейну, и превращаю в матрицу
data_test = dataset.build_interactions(X[1])[1] #проделываю то же самое для теста

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [18]:
mlightFM = LightFM(k=10, loss = 'warp')
mlightFM.fit(data_train, epochs = 100)

In [19]:
precision_at_k(mlightFM, data_test, data_train, 10).mean()

0.0068228105

Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [20]:
df_new = pd.DataFrame(interactions_full_df.contentId.unique(), columns = ['contentId'])

In [21]:
adf = articles_df.copy()
ff = pd.merge(df_new, adf, on = 'contentId', how = 'left' ) #создаю датафрейм из статей, ранжированных через контентайд из датафрейма с интеракциями
ff['text'] = ff['text'].fillna('no text') #здесь просто пытаюсь заполнить пропуск хоть чем-то

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tr = TfidfVectorizer()

feat = tr.fit_transform(ff.text)

In [23]:
mlightFM2 = LightFM(k=10, loss = 'warp')
mlightFM2.fit(data_train, epochs = 100, item_features = feat)

In [24]:
precision_at_k(mlightFM2, data_test, data_train, 10, item_features = feat).mean()

0.0067209774

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [25]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
#создаю новый датафрейм, включаю в него данные по айди, языку и, собственно, тексту статьи (заголовок решила опустить), заполняю пропуски в столбцах
adf_new = articles_df.copy()
adf_new['text'] = articles_df['text']
adf_new = adf_new[['contentId','lang','text']]
adf_new = pd.merge(df_new, adf_new, on = 'contentId', how = 'left' )
adf_new['text'] = adf_new['text'].fillna('unknown')
adf_new['lang'] = adf_new['lang'].fillna('no text')
adf_new.head()

,contentId,lang,text
0,-5065077552540450930,pt,A AXA se manteve na liderança do ranking de ma...
1,-6623581327558800021,en,"About a decade ago, a handful of Google's most..."
2,-793729620925729327,en,"Posted by Sam Thorogood , Developer Programs E..."
3,1469580151036142903,en,This is one of the great discussions among dev...
4,7270966256391553686,en,We are excited to announce the release of .NET...


In [27]:
adf_new.lang.value_counts() 

en         2148
pt          822
no text       8
la            2
es            2
ja            2
Name: lang, dtype: int64

In [28]:
print(stopwords.fileids())

['arabic', 'azerbaijani', 'bengali', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


nltk предлагает возможность взять готовый набор стоп-слов для английского, португальского и испанского
испанский было принято решение не включать в набор стоп слов ввиду малого количества айтемов на испанском языке и потенциального пересечения испанских стоп-слов с португальскими качественными словами (хотя я пыталась, но это только ухудшало качество)
латынь и японский за неимением готового набора стоп-слов at hand и малым количеством наблюдений остаются без стоп слов и очищаются только от пунктуации

In [83]:
en = stopwords.words('english') + list(punctuation) 
pt = stopwords.words('portuguese') + list(punctuation) 
#es = stopwords.words('spanish') + list(punctuation)

In [30]:
#!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 3.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=9877cd8843ed383239788b73319212b7313fea4507a55ad25ff281057c629c1a
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [104]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from langdetect import detect

lmtzr = WordNetLemmatizer()
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#я решила попробовать по-разному работать с данными английского и португальского языка
#поэтому португальский язык подвергаю стеммингу при помощи стеммера, который лучше подходит для испанского, а английский - леммизирую
def my_tokenizer(x):
    wt = word_tokenize(x)  
    if detect(x) == 'pt':  
        preprocessed = [stemmer.stem(word) for word in wt if word not in pt and word.isalpha()]  
    #elif detect(x) == 'es':
        #preprocessed = [SnowballStemmer('spanish').stem(word) for word in wt if word not in es and word.isalpha()]  
        #тут была  попытка процессить испанский тоже, но она ухудшала качество... модель выдавала не больше 0.006 precision@k
    else:    
        preprocessed = [lmtzr.lemmatize(word) for word in wt if word not in en and word.isalpha()]
        #preprocessed = [SnowballStemmer('english').stem(word) for word in wt if word not in noise_en and word.isalpha()]
        #тут тоже запечатлеваю разные пробы пера в стемминге, но остановилась именно на лемматизации, а не стемминге с английским языком
    return preprocessed

In [92]:
stopwords_all = stopwords.words('portuguese') + list(punctuation) + stopwords.words('english') #+ stopwords.words('spanish')
tr2 = TfidfVectorizer(lowercase = True, tokenizer = my_tokenizer, stop_words = stopwords_all)
feat2 = tr2.fit_transform(adf_new['text'])

In [93]:
mlightFM3 = LightFM(k=10, loss = 'warp')
mlightFM3.fit(data_train, item_features = feat2, epochs = 100)

In [94]:
precision_at_k(mlightFM3, data_test, data_train, item_features = feat2, k = 10).mean()

0.008350305

В сравнении с предыдущими моделями, качество улучшилось, однако незначительно - на доли процента. Предполагаю, что низкое значение precision@k может быть связано с отсутствием заголовка в тексте статьи, который мог бы позволить лучше обучаться. Рост качества может быть ограничен тем, что стеммингу не подвергаются японские и латинские айтемы, но, скорее всего, незначительно, поскольку в общей выборке их мало.

Подберем гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [37]:
import itertools



def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(10, 200), #чуть увеличила размах в сравнении с предложенным на сайте
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),  
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-7),
            "user_alpha": np.random.exponential(1e-7),
            "max_sampled": np.random.randint(5, 50),
            "num_epochs": np.random.randint(5, 50),
        }

def random_search(train, test, num_samples=10, num_threads=1):
    """
    Sample random hyperparameters, fit a LightFM model, and evaluate it
    on the test set.

    Parameters
    ----------

    train: np.float32 coo_matrix of shape [n_users, n_items]
        Training data.
    test: np.float32 coo_matrix of shape [n_users, n_items]
        Test data.
    num_samples: int, optional
        Number of hyperparameter choices to evaluate.


    Returns
    -------

    generator of (precision, hyperparameter dict, fitted model)

    """

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams, k = 10, random_state = 0) 
        model.fit(train, epochs = num_epochs, num_threads=num_threads)

        score = precision_at_k(model, test, train, k = 10, num_threads=num_threads).mean()

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

In [99]:
(score, hyperparams, model) = max(random_search(data_train, data_test, num_threads = 2), key=lambda x: x[0])
print("Best score {} at {}".format(score, hyperparams))

Best score 0.0070264763198792934 at {'no_components': 28, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.019975266458565906, 'item_alpha': 7.498438381490262e-09, 'user_alpha': 6.665559783517674e-09, 'max_sampled': 27, 'num_epochs': 35}


In [79]:
m_final = LightFM(random_state = 0,
                   no_components = 100, 
                   learning_schedule = 'adagrad', 
                   loss = 'warp', 
                   learning_rate = 0.025, 
                   item_alpha =  1.4921059666702605e-09, 
                   user_alpha =  2.1366172295379337e-08,
                   max_sampled = 19)
m_final.fit(data_train, epochs = 26)

#немного видоизменяла параметры, предлагаемые механизмом подбора выше, в итоге остановилась на этих

In [80]:
precision_at_k(m_final, data_test, data_train, k = 10).mean()

0.008655804

In [81]:
m_final.fit(data_train, item_features = feat, epochs = 26)
precision_at_k(m_final, data_test, data_train, item_features = feat, k = 10).mean()

0.008757638

In [100]:
m_final.fit(data_train, item_features = feat2, epochs = 26)
precision_at_k(m_final, data_test, data_train, item_features = feat2, k = 10).mean()

0.008655804

Лучшее качество получилось на модели с tf-idf но без серьезной предобработки
около 0,0087